

# Introduction

## Overview
### Plan

At the end of this session, you will have learned how to:

* Take advantage of the verbs and syntax you learned from the `dplyr` module to manipulate `RxXdfData` data objects
* Summarize your `RxXdfData` objects quickly and easily
* Create custom functions and use them for mutations and summarizations
* Understand where and when to use the `dplyrXdf` package and when to use functions from the `RevoScaleR` package

## Why dplyrXdf?
### Simplify Your Analysis Pipeline

* The `RevoScaleR` package enables R users to manipulate data that is larger than memory
* It introduces a new data type, called an `xdf` (short for eXternal Data Frame), which are highly efficient out-of-memory objects
* However, many of the `RevoScaleR` functions have a dramatically different syntax from base R functions
* The `dplyr` package is an exceptionally popular, due to its appealing syntax, and it's extensibility

## Simpler Analysis with dplyrXdf

* The `dplyrXdf` that exposes most of the `dplyr` functionality to `xdf` objects
* Many data analysis pipelines require creating many intermediate datasets, which are only needed for their role in deriving a final dataset, but have no/little use on their own
* The `dplyrXdf` abstracts this task of file management, so that you can focus on the data itself, rather than the management of intermediate files
* Unlike `dplyr`, or other base R packages, `dplyrXdf` allows you to work with data residing _outside_ of memory, and therefore scales to datasets of arbitrary size


## Requirements
### What You'll Need

* I expect that you have already covered the `dplyr` training
* Understand the *XDF* data type and how to import data to *XDF*
* If you're working on a different computer than your trianer: have (`devtools`)[github.com/hadley/devtools] (and if on a Windows machine, [Rtools](https://cran.r-project.org/bin/windows/Rtools/))

## Installing dplyrXdf

* The `dplyrXdf` package is not yet on CRAN
* You have to download it from [github](https://github.com/RevolutionAnalytics/dplyrXdf/)
  - if you're on a windows machine, install [Rtools](https://cran.r-project.org/bin/windows/Rtools/) as well
  - the `devtools` package provides a very handy function, `install_github`, for installing R packages saved in github repositories

## Create XDF from taxi data

### Create a local directory to save XDF



In [ ]:
#----------------------
# Code cell 1
#----------------------

# Set .libPaths and load libraries
.libPaths(c(.libPaths(),'/anaconda/lib/R/library/'))
library(dplyr)
library(stringr)
library(ggplot2)
library(broom)
library(dplyrXdf)


In [ ]:
#----------------------
# Code cell 2
#----------------------

# Set File Name to write to
your_name <- "YOUR NAME HERE"
your_dir <- paste0('/home/LOGIN NAME HERE/notebooks/', your_name)

# File Path to your Data
your_data <- file.path(your_dir, 'tripdata_2015.xdf')
dir.create(your_dir)
download.file("http://alizaidi.blob.core.windows.net/training/yellow_tripdata_2015.xdf",
              destfile = your_data)


## Create a Pointer to XDF



In [ ]:
#----------------------
# Code cell 3
#----------------------

taxi_xdf <- RxXdfData(your_data)
taxi_xdf %>% head
taxi_xdf %>% nrow
class(taxi_xdf)



# Simplified Pipelines for Data Summaries

## Data Transforms
### The rxDataStep Way

* All the functionality exposed by the `dplyrXdf` package can also be completed
by using the `rxDataStep` function in the `RevoScaleR` package included with your MRS installation
* In fact, `dplyrXdf` consists almost entirely of wrapper functions that call on other RevoScaleR functions
* Let's compare the workflow for adding a new column to a dataset with `rxDataStep` vs `dplyrXdf`

---



In [ ]:
#----------------------
# Code cell 4
#----------------------

taxi_xdf %>% rxGetInfo(getVarInfo = TRUE, numRows = 4)


---


In [ ]:
#----------------------
# Code cell 5
#----------------------

taxi_transform <- RxXdfData(your_data)


---




In [ ]:
#----------------------
# Code cell 6
#----------------------

system.time(rxDataStep(inData = taxi_xdf,
           outFile = taxi_transform,
           transforms = list(tip_pct = tip_amount/fare_amount),
           overwrite = TRUE))


## Data Transforms
### The rxDataStep Way



In [ ]:
#----------------------
# Code cell 7
#----------------------

rxGetInfo(RxXdfData(taxi_transform), numRows = 2)


## Data Transforms
### The dplyrXdf Way

* We could do the same operation with `dplyrXdf`, using the exact same syntax
that we learned in the `dplyr` module and taking advantage of the `%>%` operator



In [ ]:
#----------------------
# Code cell 8
#----------------------

system.time(taxi_transform <- taxi_xdf %>% mutate(tip_pct = tip_amount/fare_amount))
taxi_transform %>% rxGetInfo(numRows = 2)

## Differences
* The major difference between the rxDataStep operation and the dplyrXdf method, is that we do not specify an outFile argument anywhere in the dplyrXdf pipeline
* In our case, we have assigned our mutate value to a new variable called taxi_transform
* This creates a temporary file to save the intermediate xdf, and only saves the most recent output of a pipeline, where a pipeline is defined as all operations starting from a raw xdf file.
* Temporary files are deleted when you quit R.  See [dplyrXdf documentation](https://github.com/RevolutionAnalytics/dplyrXd) for details. 
* To copy an xdf from the temporary directory to permanent storage, use the _persist_ verb


In [ ]:
#----------------------
# Code cell 9
#----------------------

taxi_transform@file
persist(taxi_transform, outFile = paste0(your_dir,"/taxiTransform.xdf")) -> taxi_transform


## Using dplyrXdf for Aggregations
### dplyrXdf Way

* The `dplyrXdf` package really shines when used for data aggregations and summarizations
* Whereas `rxSummary`, `rxCube`, and `rxCrossTabs` can compute a few summary statistics and do aggregations very quickly, they are not sufficiently general to be used in all places
---


In [ ]:
#----------------------
# Code cell 10
#----------------------

taxi_group <- taxi_transform %>% group_by(pickup_nhood) %>% summarize(Count = n())

taxi_group %>% head


## Using dplyrXdf for Aggregations
### rxCube Way

* The above could have been done with `rxCube` as well, but would require additional considerations
* We would have to make sure that the `pickup_nhood` column was a factor (can't mutate in place because of different data types)
* `rxCube` can only provide summations and averages, so we cannot get standard deviations for instance.
* Creating your own factors is never a pleasant experience. You may feel like everything is going right until

![faceplant](http://www.ohmagif.com/wp-content/uploads/2015/02/dude-front-flip-epic-face-plant.gif)
---


In [ ]:
#----------------------
# Code cell 11
#----------------------

rxFactors(inData = taxi_transform,
          outFile = paste0(your_dir,"/taxi_factor.xdf"),
          factorInfo = c("pickup_nhood"),
          overwrite = TRUE)

head(rxCube(tip_pct ~ pickup_nhood,
            means = TRUE,
            data = paste0(your_dir,"/taxi_factor.xdf")))



# Creating Functional Pipelines with dplyrXdf
As we saw above, it's pretty easy to create a summarization or aggregation script. We can encapsulate our aggregation into it's own function.
Suppose we wanted to calculate average tip as a function of dropoff and pickup neighborhoods. In the `dplyr` nonmenclature, this means grouping by dropoff and pickup neighborhoods, and summarizing/averaging tip percent.



In [ ]:
#----------------------
# Code cell 12
#----------------------

rxGetInfo(taxi_transform, numRows = 5)

---


In [ ]:
#----------------------
# Code cell 13
#----------------------

mht_url <- "http://alizaidi.blob.core.windows.net/training/manhattan.rds"
manhattan_hoods <- readRDS(gzcon(url(mht_url)))

---


In [ ]:
#----------------------
# Code cell 14
#----------------------

taxi_transform %>%
    filter(pickup_nhood %in% mht_hoods,
           dropoff_nhood %in% mht_hoods,
           .rxArgs = list(transformObjects = list(mht_hoods = manhattan_hoods))) %>%
    group_by(dropoff_nhood, pickup_nhood) %>%
    summarize(ave_tip = mean(tip_pct),
              ave_dist = mean(trip_distance)) %>%
    filter(ave_dist > 3, ave_tip > 0.05) -> sum_df




---



In [ ]:
#----------------------
# Code cell 15
#----------------------

sum_df %>% rxGetInfo(getVarInfo = TRUE, numRows = 5)
class(sum_df)


---

Alternatively, we can encapsulate this script into a function, so that we can easily call it in a functional pipeline.



In [ ]:
#----------------------
# Code cell 16
#----------------------

taxi_hood_sum <- function(taxi_data = taxi_df, ...) {

  taxi_data %>%
    filter(pickup_nhood %in% manhattan_hoods,
           dropoff_nhood %in% manhattan_hoods, ...) %>%
    group_by(dropoff_nhood, pickup_nhood) %>%
    summarize(ave_tip = mean(tip_pct),
              ave_dist = mean(trip_distance)) %>%
    filter(ave_dist > 3, ave_tip > 0.05) -> sum_df

  return(sum_df)

}



---

The resulting summary object isn't very large (about 408 rows in this case), so it shouldn't cause any memory overhead issues if we covert it now to a `data.frame`. We can plot our results using our favorite plotting library.



In [ ]:
#----------------------
# Code cell 17
#----------------------

tile_plot_hood <- function(df = taxi_hood_sum()) {


  ggplot(data = df, aes(x = pickup_nhood, y = dropoff_nhood)) +
    geom_tile(aes(fill = ave_tip), colour = "white") +
    theme_bw() +
    theme(axis.text.x = element_text(angle = 45, hjust = 1),
          legend.position = 'bottom') +
    scale_fill_gradient(low = "white", high = "steelblue") -> gplot

  return(gplot)
}


---



In [ ]:
#----------------------
# Code cell 18
#----------------------

taxi_transform <- taxi_xdf %>% mutate(tip_pct = tip_amount/fare_amount)
sum_df <- taxi_hood_sum(taxi_transform,
                        .rxArgs = list(transformObjects = list(manhattan_hoods = manhattan_hoods))) %>%
  persist(paste0(your_dir,"Summarized.xdf"))



In [ ]:
#----------------------
# Code cell 19
#----------------------

glimpse(sum_df)


# Split and Combining Operations with doXdf

## Custom functions across groups

The `do` verb is an exception to the rule that dplyrXdf verbs write their output as xdf files. This is because do executes arbitrary R code, and can return arbitrary R objects; while a data frame is capable of storing these objects, an xdf file is limited to character and numeric vectors only.

## Custom functions across groups

The doXdf verb is similar to do, but where do splits its input into one data frame per group, doXdf splits it into one xdf file per group. This allows do-like functionality with grouped data, where each group can be arbitrarily large. The syntax for the two functions is essentially the same, although the code passed to doXdf must obviously know how to handle xdfs.

---



In [ ]:
#----------------------
# Code cell 20
#----------------------

taxi_models <- taxi_xdf %>% group_by(pickup_dow) %>% doXdf(model = rxLinMod(tip_amount ~ fare_amount, data = .))

In [ ]:
#----------------------
# Code cell 21
#----------------------

taxi_models
taxi_models$model[[1]]



## Memory Issues

All the caveats that go with working with `data.frames` apply here. While each grouped partition is it's own `RxXdfData` object, the return value must be a `data.frame`, and hence, must fit in memory.
Moreover, the function you apply against the splits will determine how they are operated. If you use an `rx` function, you'll get the nice fault-tolerant, parallel execution strategies the `RevoScaleR` package provides, but for any vanilla/CRAN function will work with data.frames and can easily cause your session to crash.

---



In [ ]:
#----------------------
# Code cell 22
#----------------------

taxi_broom <- taxi_xdf %>% group_by(pickup_dow) %>% doXdf(model = lm(tip_amount ~ fare_amount, data = .))

Now we can apply the `broom::tidy` function at the row level to get summary statistics:



In [ ]:
#----------------------
# Code cell 23
#----------------------

tbl_df(taxi_broom) %>% tidy(model)